In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing, model_selection
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


2023-03-16 18:25:55.145394: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-03-16 18:25:59.248241: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-16 18:25:59.279876: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-16 18:25:59.280284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
data = pd.read_csv("pok.csv")
data.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [3]:
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(data[["Type 1", "Type 2"]]))
cat = enc.categories_[0]
def enc(arr):
    ret = np.zeros((len(arr),len(cat)))
    for elem in range(len(arr)):
        if pd.isna(arr[elem]):
            continue
        ret[elem][cat.tolist().index(arr[elem])] = 1
    return ret

In [4]:
y=enc(data["Type 1"]) + enc(data["Type 2"])
y = y.astype('int32')
y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [29]:
y_dec = np.zeros((len(y), 1), dtype="int32")
for row in range(len(y)):
    for i in range(len(y[row])):
        y_dec[row]+=y[row][i]*(2**i)
y_dic = {0:0}
for i in y_dec:
    if int(i) in y_dic:
        i=y_dic[int(i)]
    else:
        y_dic[int(i)]=list(y_dic.values())[len(list(y_dic.values()))-1]+1
        i = y_dic[int(i)]
for i in range(len(y_dec)):
    y_dec[i] = y_dic[int(y_dec[i])]
len(y_dic)

134

In [28]:
x = np.array(data.drop(["#", "Name", "Type 1", "Type 2", "Generation", "Legendary"], 1))
print(x.shape)
print(x[:3])
#y = np.array(data["Star type"], dtype='float')
y.shape = (len(y),18)
scaler = preprocessing.MinMaxScaler().fit(x)
x = scaler.transform(x)
#x = x.reshape(x.shape[0], x.shape[1], 1)
print(x[0])


(800, 7)
[[318  45  49  49  65  65  45]
 [405  60  62  63  80  80  60]
 [525  80  82  83 100 100  80]]
[0.23       0.17322835 0.23783784 0.19555556 0.29891304 0.21428571
 0.22857143]


/tmp/ipykernel_2543/744984640.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = np.array(data.drop(["#", "Name", "Type 1", "Type 2", "Generation", "Legendary"], 1))


In [47]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y_dec, test_size=0.3)

In [48]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, input_shape=(7,), activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(134, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [52]:
# Train the model
model.fit(x_train, y_train, epochs=1000, batch_size=32)

Epoch 1/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.1971 - accuracy: 0.9321
Epoch 2/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.1467 - accuracy: 0.9446
Epoch 3/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.1539 - accuracy: 0.9500
Epoch 4/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.0981 - accuracy: 0.9571
Epoch 5/1000
18/18 [==============================] - 0s 4ms/step - loss: 0.0768 - accuracy: 0.9696
Epoch 6/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.0555 - accuracy: 0.9786
Epoch 7/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.0467 - accuracy: 0.9768
Epoch 8/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.0742 - accuracy: 0.9750
Epoch 9/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.0408 - accuracy: 0.9786
Epoch 10/1000
18/18 [==============================] - 0s 5ms/step - loss: 0.0363 - accuracy: 0.9786

In [40]:
score,acc = model.evaluate(x_test,y_test)

8/8 [==============================] - 0s 4ms/step - loss: 51.5886 - accuracy: 0.1250


In [41]:
print(y[:3])
model.predict(x[:3])


[[0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0]]
1/1 [==============================] - 0s 42ms/step


array([[1.35073695e-26, 9.91374791e-01, 2.54874749e-07, 0.00000000e+00,
        0.00000000e+00, 3.26224929e-03, 9.87683079e-31, 1.59390762e-12,
        1.80964423e-12, 4.88433589e-30, 4.56218896e-10, 0.00000000e+00,
        6.61780277e-15, 0.00000000e+00, 0.00000000e+00, 3.05119853e-11,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.02089119e-13,
        0.00000000e+00, 1.09570159e-03, 5.40026995e-20, 0.00000000e+00,
        6.58041892e-23, 1.09777684e-34, 0.00000000e+00, 0.00000000e+00,
        3.05442301e-27, 1.54736772e-04, 1.93524174e-09, 4.40581160e-33,
        8.13341302e-18, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.23164406e-26, 0.00000000e+00,
        0.00000000e+00, 2.31045114e-34, 0.00000000e+00, 2.38361994e-19,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.87431204e-20,
        3.07989284e-13, 1.29786668e-11, 5.18170417e-31, 1.14402192e-05,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000